Task: 
C als доделаем полноценный ml эксперимент:

1) Оптимизация гиперпараметров модели: (количество факторов и alpha - которая умножается на рейтинг) по recall@100 (графики зависимости качества от количество факторов и alpha)
2) Обучение модели с лучшими параметрами
3) Предсказание на тестовых пользователях: top-100 фильмов-рекомендаций для каждого и снятие метрик: recall@k, precision@k (в идеале еще map@k или ndcg@k). Отчет - графики этих метрик, по оси x - k, по y - значения метрик. Или просто таблица.

То есть надо сначала разбить датасет на train, test. Потом на трейне создать валидационные фолды. После этого провести эксперименты. Подумай насчет того как правильно разбивать на трейн тест в задаче рекомендаций и погугли, как это делать. На следующей неделе можем созвонится, обсудить результаты или само задание подробней.

Есть базовый, очень маленький курс по рекам от ODS и MTS можешь его глянуть, чтобы финально разобраться с мелкими нюансами.

In [1]:
import pandas as pd
import implicit
import scipy.sparse as sparse
import random
import numpy as np
from tqdm.notebook import tqdm

from itertools import islice, cycle

In [2]:
rnames = list(str.split('UserID::MovieID::Rating::Timestamp','::'))
ratings=pd.read_table('../data/ratings.dat',sep='::',header=None, names=rnames)

/Users/daniilzhilin/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
unames = list(str.split('UserID::Gender::Age::Occupation::Zip-code',"::"))
users = pd.read_table('../data/users.dat', sep='::', header=None, names=unames)

mnames = list(str.split('MovieID::Title::Genres',"::"))
movies = pd.read_csv('../data/movies.dat',sep='::',encoding='ISO-8859-1',header=None,names=["MovieID", "Title", "Genres"])

In [5]:
genres = movies.Genres.str.split('|', expand=True)

In [6]:
movies['GenreOne'] = pd.DataFrame(genres)[0]
movies['GenreTwo'] = pd.DataFrame(genres)[1]
movies['GenreThree'] = pd.DataFrame(genres)[2]
movies['GenreFour'] = pd.DataFrame(genres)[3]
movies['GenreFive'] = pd.DataFrame(genres)[4]

In [8]:
movies.drop(['Genres'], axis=1, inplace=True)

In [10]:
ratings['date'] = pd.to_datetime(ratings.Timestamp, unit='s')

In [11]:
ratings

,UserID,MovieID,Rating,Timestamp,date
0,1,1193,5,978300760,2000-12-31 22:12:40
1,1,661,3,978302109,2000-12-31 22:35:09
2,1,914,3,978301968,2000-12-31 22:32:48
3,1,3408,4,978300275,2000-12-31 22:04:35
4,1,2355,5,978824291,2001-01-06 23:38:11
...,...,...,...,...,...
1000204,6040,1091,1,956716541,2000-04-26 02:35:41
1000205,6040,1094,5,956704887,2000-04-25 23:21:27
1000206,6040,562,5,956704746,2000-04-25 23:19:06
1000207,6040,1096,4,956715648,2000-04-26 02:20:48


In [46]:
ratings.date.min()

Timestamp('2000-04-25 23:05:32')

In [13]:
ratings.date.max()

Timestamp('2003-02-28 17:49:50')

In [35]:
ratings_after_2002_05_05 = ratings[ratings.date > '2002-05-05 00:00:00']

In [36]:
ratings_after_2002_05_05.UserID.nunique()

438

In [37]:
ratings.UserID.nunique()

6040

In [38]:
train = ratings[ratings.date < '2002-05-05 00:00:00']
test = ratings_after_2002_05_05

In [40]:
sparse_item_user = sparse.csr_matrix((train['Rating'].astype(float), (train['MovieID'], train['UserID'])))
sparse_user_item = sparse.csr_matrix((train['Rating'].astype(float), (train['UserID'], train['MovieID'])))

In [42]:
!export MKL_NUM_THREADS=1

In [43]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [ ]:
model.

In [44]:
model.fit(sparse_item_user)

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
model.recommend(sparse_user_item)

TypeError: recommend() missing 1 required positional argument: 'user_items'

<h3> Делаем разбиение по дате </h3>